# Model 1

This is simple CNN Model for training on Rp image's

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
from torch.optim.lr_scheduler import ReduceLROnPlateau
import random
import numpy as np
from tqdm import tqdm
from collections import defaultdict
from sklearn.utils.class_weight import compute_class_weight
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch.nn import TransformerEncoder, TransformerEncoderLayer

## initiation:

In [4]:
"""
from Prop.16
"""

import os
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader, random_split

class TensorDataset(Dataset):  # squeezed for adding one channel tensor
    def __init__(self, root_dir):
        self.root_dir = root_dir
        self.classes = sorted(os.listdir(root_dir))
        self.files = []
        for class_idx, class_name in enumerate(self.classes):
            class_path = os.path.join(root_dir, class_name)
            if os.path.isdir(class_path):
                for file_name in os.listdir(class_path):
                    file_path = os.path.join(class_path, file_name)
                    if file_name.endswith(".pt"):
                        self.files.append((file_path, class_idx))

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        file_path, label = self.files[idx]
        tensor = torch.load(file_path)  # Load the tensor
        tensor = tensor.unsqueeze(0)   # Add channel dimension: [1, 160, 160]
        return tensor, label


# Path to your dataset
patheon = r"F:\TheLis\4_Class"

# Load the dataset
dataset = TensorDataset(root_dir=patheon)

# Define the train/test split ratio
train_size = int(0.9 * len(dataset))
test_size = len(dataset) - train_size

# Randomly split the dataset
train_dataset, test_dataset = random_split(dataset, [train_size, test_size])

# Define DataLoaders for both training and test sets
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# Verify the DataLoader
for tensors, labels in train_loader:
    print("Batch tensors shape:", tensors.shape)
    print("Batch labels:", labels)
    break

Batch tensors shape: torch.Size([64, 1, 160, 160])
Batch labels: tensor([1, 1, 2, 0, 1, 1, 2, 1, 2, 0, 2, 2, 0, 3, 1, 1, 0, 2, 1, 1, 2, 3, 1, 3,
        1, 2, 0, 1, 1, 0, 3, 2, 2, 2, 2, 3, 1, 2, 2, 0, 3, 0, 2, 2, 2, 0, 0, 3,
        3, 3, 1, 3, 0, 0, 2, 1, 1, 3, 1, 1, 3, 2, 1, 1])


C:\Users\Admin\AppData\Local\Temp\ipykernel_8076\2620370336.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tensor = torch.load(file_path)  # Load the tensor


## Model:

In [5]:
class ImprovedCNNWithTransformer2(nn.Module):
    def __init__(self, num_classes=4, num_transformer_layers=4, num_heads=8):
        super().__init__()

        # Convolutional Layers (updated for single-channel input)
        self.conv1 = nn.Conv2d(1, 16, kernel_size=3, stride=1, padding=1)  # 1 -> 16
        self.conv2 = nn.Conv2d(16, 32, kernel_size=3, stride=1, padding=1)  # 16 -> 32
        self.conv3 = nn.Conv2d(32, 64, kernel_size=3, stride=2, padding=1)  # 32 -> 64
        self.conv4 = nn.Conv2d(64, 128, kernel_size=3, stride=2, padding=1)  # 64 -> 128 (New Layer)

        self.shortcut = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=1, stride=2),  # Match dimensions for residual connection
            nn.BatchNorm2d(128)
        )

        self.bn1 = nn.BatchNorm2d(16)  # Updated BatchNorm layers
        self.bn2 = nn.BatchNorm2d(32)
        self.bn3 = nn.BatchNorm2d(64)
        self.bn4 = nn.BatchNorm2d(128)

        # Transformer Parameters
        self.embed_dim = 128  # Keep embedding size to match final convolution output
        self.global_pool = nn.AdaptiveAvgPool2d((4, 4))  # Fixed-size output for tokenization

        # Transformer Encoder
        encoder_layer = TransformerEncoderLayer(d_model=self.embed_dim, nhead=num_heads, dim_feedforward=512, dropout=0.4, batch_first=True)
        self.transformer = TransformerEncoder(encoder_layer, num_layers=num_transformer_layers)

        # Fully Connected Layers
        self.fc1 = nn.Linear(self.embed_dim, 1024)
        self.fc2 = nn.Linear(1024, num_classes)
        self.dropout = nn.Dropout(0.4)

    def forward(self, x):
        # Block 1
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.max_pool2d(x, 2)

        # Block 2
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.max_pool2d(x, 2)

        # Block 3
        x = F.relu(self.bn3(self.conv3(x)))
        x = F.max_pool2d(x, 2)               #### i add myself

        # Block 4 with Residual
        shortcut = self.shortcut(x)  # Downsample shortcut
        x = F.relu(self.bn4(self.conv4(x)) + shortcut)

        # Global Pooling
        x = self.global_pool(x)  # Shape: [batch_size, 128, 4, 4]
        batch_size, channels, height, width = x.size()

        # Prepare Transformer Input
        x = x.view(batch_size, channels, -1).permute(0, 2, 1)  # Shape: [batch, 16, 128]

        # Transformer Encoder
        x = self.transformer(x)  # Shape: [batch, 16, 128]
        x = x.mean(dim=1)  # Aggregate token representations (Shape: [batch, 128])

        # Fully Connected Layers
        x = self.dropout(F.relu(self.fc1(x)))
        x = self.fc2(x)
        return x
model = ImprovedCNNWithTransformer2()

# eval to Model Work right of not ?
input_tensor = torch.randn(64, 1, 160, 160)  # Batch size 8, single channel, 160x160 resolution
output = model(input_tensor)
print(output.shape)  # Expected: [8, num_classes]

torch.Size([64, 4])


## Eval Section:

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

criterion = nn.CrossEntropyLoss()
optimizer = optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-2)
num_epochs = 70  # Assuming a total of 50 epochs
model.to(device)

# Variable to track the best model
best_test_accuracy = 0.0
best_model_path = "best_model.pth"

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0

    # Training loop
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        correct_predictions += (predicted == labels).sum().item()
        total_samples += labels.size(0)

    # Calculate training metrics
    epoch_loss = running_loss / len(train_loader)
    epoch_accuracy = correct_predictions / total_samples
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, "
          f"Accuracy: {epoch_accuracy:.2%}, "
          f"LR: {optimizer.param_groups[0]['lr']:.6f}")

    # Start testing after 40th epoch
    if epoch >= 19:
        model.eval()
        correct_predictions = 0
        total_samples = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                correct_predictions += (predicted == labels).sum().item()
                total_samples += labels.size(0)

        # Calculate test accuracy
        test_accuracy = correct_predictions / total_samples
        print(f"Test Accuracy after Epoch {epoch+1}: {test_accuracy:.2%}")

        # Save the model if it's the best so far
        if test_accuracy > best_test_accuracy:
            best_test_accuracy = test_accuracy
            torch.save(model.state_dict(), best_model_path)
            print(f"Best model saved with accuracy: {best_test_accuracy:.2%}")

# Load the best model for further use
print(f"Training complete. Best model accuracy: {best_test_accuracy:.2%}")
model.load_state_dict(torch.load(best_model_path))

C:\Users\Admin\AppData\Local\Temp\ipykernel_8076\2620370336.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tensor = torch.load(file_path)  # Load the tensor


Epoch [1/70], Loss: 1.0769, Accuracy: 51.80%, LR: 0.000300
Epoch [2/70], Loss: 0.9223, Accuracy: 58.39%, LR: 0.000300
Epoch [3/70], Loss: 0.7882, Accuracy: 64.83%, LR: 0.000300
Epoch [4/70], Loss: 0.7113, Accuracy: 69.46%, LR: 0.000300
Epoch [5/70], Loss: 0.6921, Accuracy: 69.88%, LR: 0.000300
Epoch [6/70], Loss: 0.6204, Accuracy: 73.31%, LR: 0.000300
Epoch [7/70], Loss: 0.5862, Accuracy: 75.02%, LR: 0.000300
Epoch [8/70], Loss: 0.5601, Accuracy: 77.63%, LR: 0.000300
Epoch [9/70], Loss: 0.5284, Accuracy: 78.45%, LR: 0.000300
Epoch [10/70], Loss: 0.4572, Accuracy: 81.61%, LR: 0.000300
Epoch [11/70], Loss: 0.4241, Accuracy: 82.48%, LR: 0.000300
Epoch [12/70], Loss: 0.3996, Accuracy: 83.64%, LR: 0.000300
Epoch [13/70], Loss: 0.4018, Accuracy: 83.39%, LR: 0.000300
Epoch [14/70], Loss: 0.4180, Accuracy: 83.50%, LR: 0.000300
Epoch [15/70], Loss: 0.4036, Accuracy: 83.73%, LR: 0.000300
Epoch [16/70], Loss: 0.3764, Accuracy: 84.44%, LR: 0.000300
Epoch [17/70], Loss: 0.3629, Accuracy: 85.35%, LR

C:\Users\Admin\AppData\Local\Temp\ipykernel_8076\1338363190.py:65: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(best_model_path))


<All keys matched successfully>

## Accuracy:

In [7]:
"""
Test section

Explanation of Metrics
Overall Test Accuracy:

Direct comparison of predicted vs. actual labels across the entire test set.
accuracy_score(all_labels, all_preds) calculates this metric.
Per-Class Accuracy:

Measures the accuracy of the model for each individual class.
Computed using the diagonal values of the confusion matrix (correct predictions per class) divided by the number of samples per class.
Classification Report:

Includes precision, recall, and F1-score for each class.
This provides a detailed understanding of how well the model performs for each class, especially in cases of class imbalance.
Confusion Matrix:

Shows the number of true positives, false positives, false negatives, and true negatives for each class.
Useful for diagnosing specific patterns of misclassification.

"""

import numpy as np
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

def evaluate_model(model, test_loader, device):
    model.eval()
    all_preds = []
    all_labels = []

    with torch.no_grad():
        for inputs, labels in tqdm(test_loader, desc="Evaluating"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)

            # Get predictions
            _, preds = torch.max(outputs, 1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    # Convert to numpy arrays for evaluation
    all_preds = np.array(all_preds)
    all_labels = np.array(all_labels)

    # Overall accuracy
    accuracy = accuracy_score(all_labels, all_preds)
    print(f"Overall Test Accuracy: {accuracy:.4f}")

    # Per-class accuracy
    class_accuracy = np.diag(confusion_matrix(all_labels, all_preds)) / np.bincount(all_labels)
    for i, acc in enumerate(class_accuracy):
        print(f"Class {i} Accuracy: {acc:.4f}")

    # Classification report
    print("\nClassification Report:")
    print(classification_report(all_labels, all_preds, target_names=dataset.classes))

    # Confusion matrix
    print("\nConfusion Matrix:")
    print(confusion_matrix(all_labels, all_preds))

    return accuracy

# Test the model
test_accuracy = evaluate_model(model, test_loader, device)

Evaluating:   0%|          | 0/8 [00:00<?, ?it/s]C:\Users\Admin\AppData\Local\Temp\ipykernel_8076\2620370336.py:28: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  tensor = to

Overall Test Accuracy: 0.7780
Class 0 Accuracy: 0.8261
Class 1 Accuracy: 0.6613
Class 2 Accuracy: 0.8480
Class 3 Accuracy: 0.7794

Classification Report:
              precision    recall  f1-score   support

           1       0.67      0.83      0.74       115
           2       0.73      0.66      0.69       124
           3       0.88      0.85      0.86       125
           4       0.85      0.78      0.82       136

    accuracy                           0.78       500
   macro avg       0.78      0.78      0.78       500
weighted avg       0.79      0.78      0.78       500


Confusion Matrix:
[[ 95  15   0   5]
 [ 35  82   2   5]
 [  1  10 106   8]
 [ 11   6  13 106]]
